In [1]:
#import needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from time import time
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
sns.set()
%matplotlib inline

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
# Total number of records
n_records = len(train)

# Number of records where the client made the transaction
made = train.target.sum()

# Number of records where the client didn't make the transaction
didnt_make = n_records - made

# Percentage of transaction made
percentage = made / n_records * 100

# Print the results
print("Total number of records: {}".format(n_records))
print("Clients that made the transaction: {}".format(made))
print("Clients that didn't make the transaction: {}".format(didnt_make))
print("Percentage of clients that made the transaction: {}%".format(percentage))

Total number of records: 200000
Clients that made the transaction: 20098
Clients that didn't make the transaction: 179902
Percentage of clients that made the transaction: 10.049%


In [5]:
#Setting up X and y
train_data = train.copy()
y = train_data.target
X = train_data.drop(['ID_code', 'target'], axis=1)

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)

In [7]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 160000 samples.
Testing set has 40000 samples.


In [8]:
from xgboost import XGBClassifier

In [9]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

#cv_results = xgb.cv(dtrain=D_train, params=params, nfold=10,
#                    num_boost_round=1000,early_stopping_rounds=50,metrics="auc", as_pandas=True, seed=123)

C:\Users\eders\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


# Training and Testing

In [10]:
def model_training(dtrain, params, rounds):
    model = xgb.train(dtrain= dtrain, params= params, num_boost_round= rounds)
    return model

In [11]:
def auc_score(dtest, y_test, model):
    preds = model.predict(dtest)
    return round(roc_auc_score(y_test, preds), 4)

# Submission

In [12]:
def transform_data(data):
    data = data.drop(['ID_code'], axis = 1)
    scaler = StandardScaler() 
    test_features = scaler.fit_transform(data)
    final_data = xgb.DMatrix(test_features)
    return final_data

In [13]:
def make_csv(test_data, model):
    preds_final = model.predict(test_data)
    predictions = pd.Series(preds_final, index = test['ID_code'], name = 'target')
    return predictions.to_csv("submission.csv", header = True)

In [26]:
for i in np.arange(4):
    params = {
    'objective': "binary:logistic", 
    'colsample_bytree': .3, 
    'learning_rate': 0.1, 
    'max_depth': 2,
    'gamma': 5,
    'subsample': 0.7 + (i * 0.05),
    'reg_alpha' : 0.4,
    'scale_pos_weight': 1,
    'tree_method': 'gpu_hist',
    'silent': 1}
    model = model_training(D_train, params, 2500)
    print("AUC = {}".format(auc_score(D_test, y_test, model)))

AUC = 0.8947
AUC = 0.8941
AUC = 0.8938
AUC = 0.8942


In [22]:
test_data = transform_data(test)
make_csv(test_data, model)